In [ ]:
from STK_Sim import *

Filename = 'AERO_402_Further_Assessment'

stk_object = STK_Simulation(False,Filename)

In [ ]:
# Setting scenario time variables
stk_object.root.UnitPreferences.SetCurrentUnit("DateFormat", "UTCG")
start_time = time_convert(stk_object.root.CurrentScenario.StartTime)
duration = datetime.timedelta(days=1, seconds=0, microseconds=0, milliseconds=0, minutes=0, hours=0, weeks=0)
stop_time=(start_time+duration).strftime("%d %b %Y %H:%M:%S.%f")
stk_object.root.CurrentScenario.StopTime=stop_time
stk_object.root.UnitPreferences.SetCurrentUnit("DateFormat", "EpSec")

# All of these variables substantially change calculation time
stk_object.dt = 30
n_targets = 15
n_sats = 12
n_pop = 10
n_gen = 5

Opt = Optimizer(stk_object,n_pop,n_gen,n_sats)

In [ ]:
# Generating Targets

# Generating a polygon to bound the lat/lon coordinates, you can create your polygon, in the same format as Targets_Polygon.txt.
poly = Create_Poly('Input_Files/Targets_Polygon.txt')

# Writing random points within the polygon to a target file.
targets_filename = 'Input_Files/Targets_File.txt'
polygon_random_points(poly,n_targets,targets_filename)

# Loading targets into stk from file.
stk_object.Target_Loader(targets_filename)

# # Plotting the polygon and generated targets on the map.
# plot_targets_and_polygon(poly,targets_filename)

In [ ]:
# Calling optimizer object and running optimization

stk_object.dt = 30
stk_object.Interpolate = True
hof,percent,times,sats = Opt.run()
Opt.cost_function(hof[0])
print(pd.DataFrame(percent))

In [ ]:
from requirements.OCD_1_1 import Check_Lifetime

for h in hof:
    print(h)
    Opt.cost_function(h)
    Check_Lifetime(stk_object).run()

In [ ]:
# Comparing high data rate w/ no interpolation to low data rate w/ interpolation data

stk_object.root.UnitPreferences.SetCurrentUnit("DateFormat", "UTCG")
start_time = time_convert(stk_object.root.CurrentScenario.StartTime)
duration = datetime.timedelta(days=30, seconds=0, microseconds=0, milliseconds=0, minutes=0, hours=0, weeks=0)
stop_time=(start_time+duration).strftime("%d %b %Y %H:%M:%S.%f")
stk_object.root.CurrentScenario.StopTime=stop_time
stk_object.root.UnitPreferences.SetCurrentUnit("DateFormat", "EpSec")

t_interp = time.time()

stk_object.dt = 30
stk_object.Interpolate = True
p1,t1,s1=Opt.cost_function(write=False)

interpolated_target_bins = stk_object.target_bins.copy()

t_interp = time.time()-t_interp

# t_sim = time.time()
# stk_object.dt = 2.5
# stk_object.Interpolate = False
# p2,t2,s2=Opt.cost_function(write=False)

# t_sim = time.time()-t_sim

# target_bins = stk_object.target_bins.copy()

In [ ]:
fig = px.imshow(np.array([bin.T[::-1] for bin in interpolated_target_bins]),animation_frame=0,zmax=1,zmin=0, color_continuous_scale='Greys')
fig.update_layout({"title":f"Interpolated Data-(comp time={t_interp:.2f}s,p={p1:.2f}%,t={t1:.4f} days,sats={s1})",
                   "xaxis_title":"Azimuth",
                   "yaxis_title":"Elevation"})
fig.show()

# fig = px.imshow(np.array([bin.T[::-1] for bin in target_bins]),animation_frame=0)
# fig.update_layout({"title":f"Simulated Data-(comp time={t_sim:.2f}s,p={p2:.2f}%,t={t2:.4f} days,sats={s2})",
#                    "xaxis_title":"Azimuth",
#                    "yaxis_title":"Elevation"})
# fig.show()

In [ ]:
import matplotlib.pyplot as plt
%matplotlib tk

N = len(stk_object.satellites)
fig,axes = plt.subplots(N//2,2)
for interval in stk_object.Intervals:
    tar = interval.target_number
    sat = interval.satellite_number
    axes[sat].plot([interval.start,interval.stop],[tar,tar],marker="|")